In [1]:
!pip install -U trl peft
!pip install -U bitsandbytes
import torch
import wandb
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments
original_dataset = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset", split="train")
def create_prompt(sample):
    return f"""<|user|>
{sample['instruction']}<|end|>
<|assistant|>
Intent: {sample['intent']}
Category: {sample['category']}
Response: {sample['response']}<|end|>"""
model_id = 'microsoft/Phi-3-mini-4k-instruct'
device = 'cuda'
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
lora_config = LoraConfig(
    r=16,  
    lora_alpha=32, 
    target_modules="all-linear", 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=False 
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token 
tokenizer.padding_side = "right"
training_arguments = SFTConfig(  
        output_dir="./phi3-tuned-chatbot",
        dataset_text_field="text",
        max_length=512,
        per_device_train_batch_size=1, 
        gradient_accumulation_steps=4, 
        learning_rate=2e-4,
        max_steps=1000,
        logging_steps=25,
    save_steps=100,
    fp16=True, # Use mixed precision
    optim="paged_adamw_8bit",
    
)
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=original_dataset,
    peft_config=lora_config,
    processing_class = tokenizer,
    formatting_func=create_prompt,
)
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
my_secret = user_secrets.get_secret("wandb_key") 
wandb.login(key=my_secret)
print("training on.........")
trainer.train()
trainer.save_model("./phi3-tuned-chatbot/final_checkpoint")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 13.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-09-21 12:23:18.228157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758457398.423766      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758457398.481898      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md: 0.00B [00:00, ?B/s]

Bitext_Sample_Customer_Support_Training_(…):   0%|          | 0.00/19.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26872 [00:00<?, ? examples/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mihirmoolchamdani (mihirmoolchamdani-dhirubhai-ambani-institute-of-informat) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


training on.........


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.182200
50,0.901100
75,0.792700
100,0.741900
125,0.763000
150,0.686500
175,0.693700
200,0.663200
225,0.686400
250,0.678100


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

In [10]:
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=False
)
tuned_model = PeftModel.from_pretrained(base_model, "./phi3-tuned-chatbot/final_checkpoint")
pipl = transformers.pipeline(
    "text-generation",
    model=tuned_model,
    tokenizer=tokenizer,
    device_map="auto"
)
user_query = "My phone is not working"
prompt = f"<|user|>\n{user_query}<|end|>\n<|assistant|>"
sequences = pipl(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
print(sequences[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


<|user|>
My phone is not working<|end|>
<|assistant|> Intent: troubleshoot_device
Category: DEVICE
Response: I'm sorry to hear that your phone is not working. I understand how frustrating it can be when technology doesn't behave as expected. Let's work together to identify the issue and find a solution. Here are a few steps you can try:

1. Check your charger and cable: Make sure that your charger and cable are properly connected and in good condition. A loose or damaged cable could cause your phone to malfunction.

2. Restart your phone: Sometimes, a simple restart can resolve the issue. Press and hold the power button on your phone, and then select the "Restart" option.

3. Update your phone's software: Check for any available software updates for your phone. Outdated software can sometimes cause compatibility issues that affect the phone's functionality.

4. Clear your phone's cache: Clearing the cache can help resolve performance issues and restore normal functioning. Go to your ph

In [ ]:
user_query = "My dog is lost"
prompt = f"<|user|>\n{user_query}<|end|>\n<|assistant|>"
sequences = pipl(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
print(sequences[0]['generated_text'])